In [1]:
import os
import pandas as pd
import h5py

%matplotlib inline

from IPython.display import display, HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
# Daysim data
trip = pd.read_csv(r'../../../outputs/daysim/_trip.tsv', sep='\t')
person = pd.read_csv(r'../../../outputs/daysim/_person.tsv', sep='\t')
hh = pd.read_csv(r'../../../outputs/daysim/_household.tsv', sep='\t')

In [3]:
def apply_labels(df,table_type):
    '''
    Replace values with human readable lablels.
    '''
    df_label = labels[labels['table'] == table_type]
    for field in df_label['field'].unique():
        newdf = df_label[df_label['field'] == field]
        local_series = pd.Series(newdf['text'].values, index=newdf['value'])
        df[field] = df[field].map(local_series)
    
    return df

In [4]:
# Add labels
labels = pd.read_csv(r'../../../scripts/summarize/inputs/calibration/variable_labels.csv')

trip = apply_labels(trip,'Trip')
person = apply_labels(person,'Person')
hh = apply_labels(hh,'Household')

In [5]:
special_taz = pd.read_csv(r'../../../scripts/summarize/inputs/special_needs_taz.csv')
rgc_taz = pd.read_csv(r'../../../scripts/summarize/inputs/rgc_taz.csv')

In [6]:
filename = r'../../../outputs/daysim/centers.xlsx'
if os.path.exists(filename):
    os.remove(filename)
writer = pd.ExcelWriter(filename)

# Mode Share to Centers by HH Location

## All Trips

In [7]:
trip_hh = pd.merge(trip,hh,on='hhno',how='left')
trip_hh = pd.merge(trip_hh, rgc_taz, left_on='hhtaz', right_on='taz', how='left')
trip_hh = trip_hh[trip_hh['mode']!='School Bus']

df = trip_hh[['trexpfac','geog_name','mode']].groupby(['geog_name','mode']).sum().reset_index()
tot_rgc = trip_hh[['trexpfac','geog_name']].groupby('geog_name').sum().reset_index()
df = pd.merge(df,tot_rgc,on='geog_name', suffixes=['_mode','_total'])
df['mode_share'] = (df['trexpfac_mode']/df['trexpfac_total'])

df = df.pivot(index='geog_name', columns='mode', values='mode_share')
# Trim the number of RGCs
rgc_list = ['Auburn','Bellevue','Bremerton','Everett','Issaquah','Lynnwood','Redmond Downtown',
       'Renton','Seattle Downtown','Seattle Northgate','Seattle University Community',
        'Seattle Uptown', 'Tacoma Downtown']


df.to_excel(writer,'HH in RGC - All Trips')
df.loc[rgc_list]

mode,Bike,HOV2,HOV3+,SOV,Transit,Walk
geog_name,,,,,,
Auburn,0.037588,0.220893,0.137906,0.339517,0.059556,0.204540
Bellevue,0.045471,0.162954,0.065652,0.251727,0.106739,0.367457
Bremerton,0.051441,0.179791,0.112651,0.306555,0.035825,0.313737
Everett,0.058984,0.167339,0.089400,0.296480,0.037927,0.349869
Issaquah,0.026101,0.168026,0.101142,0.504078,0.089723,0.110930
Lynnwood,0.045853,0.214694,0.138563,0.331443,0.059291,0.210157
Redmond Downtown,0.046650,0.170274,0.075801,0.327155,0.104190,0.275930
Renton,0.040429,0.188625,0.116161,0.323573,0.078313,0.252899
Seattle Downtown,0.085027,0.110916,0.062874,0.165120,0.149486,0.426577


## Commute Trips

In [8]:
commute_trips = trip_hh[trip_hh['dpurp'] == 'Work']

df = commute_trips[['trexpfac','geog_name','mode']].groupby(['geog_name','mode']).sum().reset_index()
tot_rgc = commute_trips[['trexpfac','geog_name']].groupby('geog_name').sum().reset_index()
df = pd.merge(df,tot_rgc,on='geog_name', suffixes=['_mode','_total'])
df['mode_share'] = (df['trexpfac_mode']/df['trexpfac_total'])

df = df.pivot(index='geog_name', columns='mode', values='mode_share')
# Trim the number of RGCs
rgc_list = ['Auburn','Bellevue','Bremerton','Everett','Issaquah','Lynnwood','Redmond Downtown',
       'Renton','Seattle Downtown','Seattle Northgate','Seattle University Community',
        'Seattle Uptown', 'Tacoma Downtown']
df = df.fillna(0)

df.to_excel(writer,'HH in RGC - Commute Trips')
df.loc[rgc_list]

mode,Bike,HOV2,HOV3+,SOV,Transit,Walk
geog_name,,,,,,
Auburn,0.064182,0.120083,0.043478,0.525880,0.074534,0.171843
Bellevue,0.058899,0.073893,0.025903,0.357749,0.155148,0.328407
Bremerton,0.101202,0.080961,0.032891,0.491461,0.042378,0.251107
Everett,0.073667,0.077258,0.036779,0.428183,0.093580,0.290533
Issaquah,0.034188,0.085470,0.008547,0.589744,0.128205,0.153846
Lynnwood,0.068625,0.092598,0.042853,0.480671,0.133653,0.181600
Redmond Downtown,0.065379,0.076445,0.030488,0.440831,0.168473,0.218383
Renton,0.062092,0.099782,0.044444,0.495425,0.115033,0.183224
Seattle Downtown,0.079159,0.061125,0.028381,0.240718,0.196626,0.393992


# Trips to Centers

## All Trips

In [9]:
trip_df = pd.merge(trip, rgc_taz, left_on='dtaz', right_on='taz', how='left')
trip_df = trip_df[trip_df['mode']!='School Bus']

df = trip_df[['trexpfac','geog_name','mode']].groupby(['geog_name','mode']).sum().reset_index()
tot_rgc = trip_df[['trexpfac','geog_name']].groupby('geog_name').sum().reset_index()
df = pd.merge(df,tot_rgc,on='geog_name', suffixes=['_mode','_total'])
df['mode_share'] = (df['trexpfac_mode']/df['trexpfac_total'])

df = df.pivot(index='geog_name', columns='mode', values='mode_share')
# Trim the number of RGCs
rgc_list = ['Auburn','Bellevue','Bremerton','Everett','Issaquah','Lynnwood','Redmond Downtown',
       'Renton','Seattle Downtown','Seattle Northgate','Seattle University Community',
        'Seattle Uptown', 'Tacoma Downtown']


df.to_excel(writer,'Trip to RGC - All Trips')
df.loc[rgc_list]

mode,Bike,HOV2,HOV3+,SOV,Transit,Walk
geog_name,,,,,,
Auburn,0.018312,0.224568,0.132383,0.468258,0.039387,0.117092
Bellevue,0.027764,0.142859,0.076951,0.287801,0.134843,0.329781
Bremerton,0.042391,0.186530,0.115251,0.374599,0.028161,0.253066
Everett,0.034878,0.183510,0.115859,0.345721,0.035170,0.284861
Issaquah,0.011519,0.203825,0.108714,0.544557,0.044027,0.087359
Lynnwood,0.029442,0.219617,0.139265,0.415311,0.036959,0.159407
Redmond Downtown,0.032465,0.196243,0.115202,0.354497,0.071727,0.229865
Renton,0.026468,0.202292,0.130742,0.406369,0.053627,0.180502
Seattle Downtown,0.044300,0.113920,0.066450,0.213509,0.208649,0.353172


## Commute Trips

In [10]:
commute_trips = trip[trip['dpurp'] == 'Work']
trip_df = pd.merge(commute_trips, rgc_taz, left_on='dtaz', right_on='taz', how='left')
trip_df = trip_df[trip_df['mode']!='School Bus']

df = trip_df[['trexpfac','geog_name','mode']].groupby(['geog_name','mode']).sum().reset_index()
tot_rgc = trip_df[['trexpfac','geog_name']].groupby('geog_name').sum().reset_index()
df = pd.merge(df,tot_rgc,on='geog_name', suffixes=['_mode','_total'])
df['mode_share'] = (df['trexpfac_mode']/df['trexpfac_total'])

df = df.pivot(index='geog_name', columns='mode', values='mode_share')
# Trim the number of RGCs
rgc_list = ['Auburn','Bellevue','Bremerton','Everett','Issaquah','Lynnwood','Redmond Downtown',
       'Renton','Seattle Downtown','Seattle Northgate','Seattle University Community',
        'Seattle Uptown', 'Tacoma Downtown']


df.to_excel(writer,'Trip to RGC - Commute Trips')
df.loc[rgc_list]

mode,Bike,HOV2,HOV3+,SOV,Transit,Walk
geog_name,,,,,,
Auburn,0.027722,0.100546,0.043809,0.645504,0.071388,0.111031
Bellevue,0.023830,0.076521,0.039002,0.391037,0.228728,0.240882
Bremerton,0.080098,0.083867,0.038070,0.566340,0.039389,0.192235
Everett,0.039616,0.086703,0.041811,0.517826,0.075791,0.238253
Issaquah,0.015927,0.093037,0.039138,0.685637,0.080412,0.085850
Lynnwood,0.043348,0.100328,0.048856,0.602582,0.063213,0.141673
Redmond Downtown,0.035561,0.090286,0.044518,0.529272,0.125387,0.174975
Renton,0.033528,0.099823,0.047431,0.601434,0.078786,0.138999
Seattle Downtown,0.040276,0.066205,0.032903,0.299109,0.285044,0.276462


In [11]:
writer.save()